In [1]:
from oct_Utils import *
from tqdm.notebook import tqdm

In [20]:
tasks = []
not_valid1 = {
    '3d wide',
#     'oct',
    'oct disc',
    'octa disc',
    'octa macula',
    'octa macular',
    'octa disc-fda',
    'octa macula-fda',
}
not_valid2 = {
    'oct',
}
for dirpath, dirnames, filenames in os.walk('/data/rawdata/OCT/data/20210730-中山眼科-OCT转VF'):
    for d in dirnames[:]:
        if '-clean-' in d or d.lower() in not_valid1:
            dirnames.remove(d)
    
    for filename in filenames:
        if filename.endswith('.fda') or filename.endswith('.fds'):
            tasks.append(f'{dirpath}/{filename}')
    print(len(tasks), end="\r", flush=True)
new_tasks = []
for task in tasks:
    parent = task.split('/')[-2]
    if parent.lower() in not_valid2:
        continue
    new_tasks.append(task)
tasks = new_tasks
print(len(tasks))
print(tasks[:10])

6588
['/data/rawdata/OCT/data/20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-097/OD/OCT-后/3D Disc/83879.fds', '/data/rawdata/OCT/data/20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-097/OD/OCT-后/3D Macula/83877.fds', '/data/rawdata/OCT/data/20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-097/OS/OCT-后/3D Disc/83884.fds', '/data/rawdata/OCT/data/20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-097/OS/OCT-后/3D Macula/83883.fds', '/data/rawdata/OCT/data/20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-143/OS/OCT-后/3D Disc/69627.fds', '/data/rawdata/OCT/data/20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-143/OS/OCT-后/3D Macula/69623.fds', '/data/rawdata/OCT/data/20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-173/OD/OCT-后/3D Disc/98685.fds', '/data/rawdata/OCT/data/20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-173/OD/OCT-后/3D Macula/98682.fds', '/data/rawdata/OCT/data/20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-173/OS/OCT-后/3D Disc/98695.fds', '/data/rawdata/OCT/data/20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-173/OS/OCT-后/3D Mac

In [21]:
json_dump(tasks, 'data/tmp.json')

In [16]:
print('OCTA Macula'.lower() in not_valid1)

True


In [30]:
def read_compacted_imgs(path):
    obj = np.load(path)
    imgs = obj['imgs']
    lens = obj['lens']
    result = []
    for img, l in zip(imgs, lens):
        img = img[:l]
        img = np.frombuffer(img, dtype=np.uint8)
        result.append(cv2.imdecode(img, cv2.IMREAD_GRAYSCALE))
    result = np.stack(result, axis=0)
    return result
tmp = read_compacted_imgs('/home/octusr2/projects/data_fast/proceeded/volume_npz/CASIA数据汇总-new/CRC-009-097/OD/OCT-后/3D Disc/83879.npz')
print(tmp.shape)

(256, 992, 512)


In [5]:
df = pd.read_csv('/home/octusr2/projects/data_fast/proceeded/volume_info/volume_info.csv')
df['oct_path'] = df['oct_path'].map(lambda x: str(Path(x).with_suffix('')))
df = df.set_index('oct_path')

print_df(df)

['oct_date_file', 'oct_x', 'oct_y', 'oct_z']
31494


,oct_date_file,oct_x,oct_y,oct_z
oct_path,,,,
20210812-sysu-oct/ANGIO1/CRC-009-106/20190730/OD/OCT/3D macula/107801,2019-07-30,6.0,6.0,2.60000
...,...,...,...,...
20210820-sysu-oct/HM-GON5/CRC000499/20210112/OD/OCT/3D macula/94968,2021-01-12,7.0,7.0,2.56361


In [5]:
input_root = Path('/home/octusr2/projects/data_fast/proceeded/volume_targz')
output_root = Path('/home/octusr2/projects/data_fast/proceeded/cp_projection/380')
info = pd.read_csv('/home/octusr2/projects/data_fast/proceeded/volume_info/volume_info.csv')
info['oct_path'] = info['oct_path'].map(lambda x: str(Path(x).with_suffix('')))
print_df(info)
tmp = info[info.duplicated(subset='oct_path', keep=False)]
tmp = tmp.sort_values(by='oct_path')
print_df(tmp.head(20), row=20)

info = info.drop_duplicates(subset='oct_path')
print_df(info)
info = info.set_index('oct_path')
print_df(info)

print(info.loc['20210812-sysu-oct/ANGIO3/CRC-009-035/20210111/OS/OCT/3D disc/115833', 'oct_x'])

filepath = input_root / '20210812-sysu-oct/ANGIO3/CRC-009-035/20210111/OS/OCT/3D disc/115833.tar.gz'
extract_cp([0, [str(filepath)]], input_root, output_root, info)

['oct_path', 'oct_date_file', 'oct_x', 'oct_y', 'oct_z']
31494


,oct_path,oct_date_file,oct_x,oct_y,oct_z
0,20210812-sysu-oct/ANGIO1/CRC-009-106/20190730/...,2019-07-30,6.0,6.0,2.60000
...,...,...,...,...,...
31493,20210820-sysu-oct/HM-GON5/CRC000499/20210112/O...,2021-01-12,7.0,7.0,2.56361


['oct_path', 'oct_date_file', 'oct_x', 'oct_y', 'oct_z']
20


,oct_path,oct_date_file,oct_x,oct_y,oct_z
14198,20210812-sysu-oct/ANGIO3/CG-0324/20210303/OD/O...,2021-03-02,6.0,6.0,2.6
1250,20210812-sysu-oct/ANGIO3/CG-0324/20210303/OD/O...,2021-03-02,6.0,6.0,2.6
4657,20210812-sysu-oct/ANGIO3/CG-0324/20210303/OD/O...,2021-03-02,6.0,6.0,2.6
26328,20210812-sysu-oct/ANGIO3/CG-0324/20210303/OD/O...,2021-03-02,6.0,6.0,2.6
26500,20210812-sysu-oct/ANGIO3/CG-0324/20210303/OS/O...,2021-03-02,6.0,6.0,2.6
2801,20210812-sysu-oct/ANGIO3/CG-0324/20210303/OS/O...,2021-03-02,6.0,6.0,2.6
3648,20210812-sysu-oct/ANGIO3/CG-0324/20210303/OS/O...,2021-03-02,6.0,6.0,2.6
29096,20210812-sysu-oct/ANGIO3/CG-0324/20210303/OS/O...,2021-03-02,6.0,6.0,2.6
27322,20210812-sysu-oct/ANGIO3/CJ-0315/20210226/OD/O...,2021-02-25,6.0,6.0,2.6
10242,20210812-sysu-oct/ANGIO3/CJ-0315/20210226/OD/O...,2021-02-25,6.0,6.0,2.6


['oct_path', 'oct_date_file', 'oct_x', 'oct_y', 'oct_z']
25584


,oct_path,oct_date_file,oct_x,oct_y,oct_z
0,20210812-sysu-oct/ANGIO1/CRC-009-106/20190730/...,2019-07-30,6.0,6.0,2.6
...,...,...,...,...,...
31492,20210812-sysu-oct/ANGIO2/ZX-0334/20170410/OD/O...,2017-04-11,3.0,3.0,2.6


['oct_date_file', 'oct_x', 'oct_y', 'oct_z']
25584


,oct_date_file,oct_x,oct_y,oct_z
oct_path,,,,
20210812-sysu-oct/ANGIO1/CRC-009-106/20190730/OD/OCT/3D macula/107801,2019-07-30,6.0,6.0,2.6
...,...,...,...,...
20210812-sysu-oct/ANGIO2/ZX-0334/20170410/OD/OCT/3D disc/75634,2017-04-11,3.0,3.0,2.6


6.0
Converting: begin
Converting, 0/1, 0.00%: 0.00 seconds
6.0
Converting: 1.33 seconds


ValueError: unknown file extension: 